In [1]:
import time
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
## Settings

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64
dropout_prob = 0.5

## Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


In [3]:
## MNIST dataset

# scale input images to 0-1 range
train_dataset = datasets.MNIST(root='../data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='../data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [7]:
## Model

class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        ## 28x28x1 => 28x28x8
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                    out_channels=8,
                                    kernel_size=(3,3),
                                    stride=(1,1),
                                    padding=1)

        ## 28x28x8 => 14x14x8
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2,2),
                                        stride=(2,2),
                                        padding=0)

        ## 14x14x8 => 14x14x16
        self.conv_2 = torch.nn.Conv2d(in_channels=8,
                                    out_channels=16,
                                    kernel_size=(3,3),
                                    stride=(1,1),
                                    padding=1)

        ## 14x14x16 => 7x7x8
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2,2),
                                        stride=(2,2),
                                        padding=0)

        self.linear_1 = torch.nn.Linear(7*7*16, num_classes)

        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
                m.weight.data.normal_(0.0, 0.1)
                m.bias.data.zero_()
                if m.bias is not None:
                    m.bias.detach().zero_()

    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)

        logits = self.linear_1(out.view(-1, 7*7*16))
        probas = F.softmax(logits, dim=1)
        return logits, probas

torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [9]:
## Training

def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float() / num_examples * 100

start_time = time.time()
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):

        features = features.to(device)
        targets = targets.to(device)

        # forward and back prop
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()

        cost.backward()

        # update model parameters
        optimizer.step()

        # logging
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 2.3120
Epoch: 001/010 | Batch 050/938 | Cost: 0.7896
Epoch: 001/010 | Batch 100/938 | Cost: 0.4248
Epoch: 001/010 | Batch 150/938 | Cost: 0.2535
Epoch: 001/010 | Batch 200/938 | Cost: 0.3390
Epoch: 001/010 | Batch 250/938 | Cost: 0.1606
Epoch: 001/010 | Batch 300/938 | Cost: 0.1062
Epoch: 001/010 | Batch 350/938 | Cost: 0.1556
Epoch: 001/010 | Batch 400/938 | Cost: 0.0945
Epoch: 001/010 | Batch 450/938 | Cost: 0.1613
Epoch: 001/010 | Batch 500/938 | Cost: 0.0986
Epoch: 001/010 | Batch 550/938 | Cost: 0.1492
Epoch: 001/010 | Batch 600/938 | Cost: 0.2650
Epoch: 001/010 | Batch 650/938 | Cost: 0.1053
Epoch: 001/010 | Batch 700/938 | Cost: 0.2458
Epoch: 001/010 | Batch 750/938 | Cost: 0.0880
Epoch: 001/010 | Batch 800/938 | Cost: 0.1634
Epoch: 001/010 | Batch 850/938 | Cost: 0.1983
Epoch: 001/010 | Batch 900/938 | Cost: 0.0735
Epoch: 001/010 training accuracy: 96.00%
Time elapsed: 0.18 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.1234
Epoch: 002/010 |

In [10]:
## Test set evaluation

with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 98.75%
